# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import pandas as pd 
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine 
import re 
import pickle


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [11]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
#Y = df.loc[:, 'related': 'direct_report']
Y = df.loc[:, 'related': 'medical_help']
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help'], dtype='object')

### 2. Write a tokenization function to process your text data

In [13]:
stemmer = PorterStemmer()
wnl = WordNetLemmatizer()
stop_words = stopwords.words('english') # stopwords are words that do not carry a lot of meanings -e.g. conjuctions, pronouns etc - they are linguistics contructions 

In [14]:
def tokenize(text):
    text_lower = text.lower()
    text_norm = re.sub(r'[^a-zA-Z0-9]', ' ', text_lower) # normalisation process 
    tokens = word_tokenize(text_norm) # tokenised words 
    tokens_clean = [wnl.lemmatize(word,pos = 'v') for word in tokens if word not in stop_words] # lemmatisation of verbs [pos = verbs]
    tokens_clean = [wnl.lemmatize(word, pos = 'n') for word in tokens_clean] # lemmatisation of nouns [pos = nouns]
    return tokens_clean 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline(
    [('vectoriser', CountVectorizer(tokenizer= tokenize)),# step 1 - creation of a doc to term matrix 
     ('transformer', TfidfTransformer()), # step 2 - Tfidf transformation 
     ('classifier', MultiOutputClassifier(MLPClassifier()))] # the classification model - i.e multioutputclassifier + RandomForestclassifier 
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:

labels = Y.columns
x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectoriser',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000026827DCAD30>)),
                ('transformer', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
def multiclass_report(true_labels, predict_labels): 
    for idx, class_column in enumerate(true_labels):
        y_true = true_labels[class_column]
        y_predict = predict_labels[:, idx]
        print(f'\n label: {class_column}\n')
        print(classification_report(y_true, y_predict))

In [ ]:
y_test_predict = pipeline.predict(x_test)
multiclass_report(y_test, y_test_predict)


 label: related

              precision    recall  f1-score   support

           0       0.70      0.36      0.47       284
           1       0.84      0.95      0.89       966

    accuracy                           0.82      1250
   macro avg       0.77      0.66      0.68      1250
weighted avg       0.80      0.82      0.80      1250


 label: request

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       643
           1       0.82      0.82      0.82       607

    accuracy                           0.82      1250
   macro avg       0.82      0.82      0.82      1250
weighted avg       0.82      0.82      0.82      1250


 label: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1249
           1       0.00      0.00      0.00         1

    accuracy                           1.00      1250
   macro avg       0.50      0.50      0.50      1250
weighted avg       1.0

c:\Github\personal\Udacity Disaster Response\Disaster_Resp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Github\personal\Udacity Disaster Response\Disaster_Resp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Github\personal\Udacity Disaster Response\Disaster_Resp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {
    'classifier__estimator__hidden_layer_sizes':[(100,)], 
    'classifier__estimator__activation':['relu',], 
    'classifier__estimator__alpha': [0.0001,]
}

cv = GridSearchCV(pipeline, parameters, scoring= 'f1_weighted')

In [19]:
cv.fit(x_train, y_train)

c:\Users\edonkoh\work\Ucity_Disaster_Response\disaster_response\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\edonkoh\work\Ucity_Disaster_Response\disaster_response\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\edonkoh\work\Ucity_Disaster_Response\disaster_response\lib\site-packages\sklearn\metrics\_scorer.py", line 219, in __call__
    return self._score(
  File "c:\Users\edonkoh\work\Ucity_Disaster_Response\disaster_response\lib\site-packages\sklearn\metrics\_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "c:\Users\edonkoh\work\Ucity_Disaster_Response\disaster_response\lib\site-packages\sklearn\metrics\_classification.py", line 1136, in f1_s

GridSearchCV(estimator=Pipeline(steps=[('vectoriser',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000029A015A4A60>)),
                                       ('transformer', TfidfTransformer()),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=MLPClassifier()))]),
             param_grid={'classifier__estimator__activation': ['relu'],
                         'classifier__estimator__alpha': [0.0001],
                         'classifier__estimator__hidden_layer_sizes': [(100,)]},
             scoring='f1_weighted')

In [20]:
pd.DataFrame(cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__estimator__activation,param_classifier__estimator__alpha,param_classifier__estimator__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4698.253975,443.84004,1.646835,0.025396,relu,0.0001,"(100,)","{'classifier__estimator__activation': 'relu', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [50]:
cv.best_params_

{'classifier__estimator__criterion': 'entropy',
 'classifier__estimator__min_samples_leaf': 4,
 'classifier__estimator__n_estimators': 200}

In [ ]:
cv.cv_results_  

In [ ]:
pd.DataFrame(cv.cv_results_).sort_values('rank_test_score')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [48]:
with open('../models/text_classifier.pickle', 'wb') as file: 
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.